# 前情
HW7的任務是模型壓縮 - Neural Network Compression。

Compression有很多種門派，在這裡我們會介紹上課出現過的其中四種，分別是:

- 知識蒸餾 Knowledge Distillation
- 網路剪枝 Network Pruning
- 用少量參數來做CNN Architecture Design
-* 參數量化 Weight Quantization

在這個notebook中我們會介紹非常簡單的Weight Quantization，而我們有提供已經做完Knowledge Distillation的小model來做Quantization。

- Model架構 / Architecute Design在同目錄中的hw7_Architecture_Design.ipynb。
 - 參數為 base=16, width_mult=1 (default)


# Weight Quantization
<img src="hw7_data/Weight Quantization.png" width="500px">

我們這邊會示範如何實作第一條: Using less bits to represent a value。

## 好的Quantization很重要。

|bit|state_dict size|accuracy|
|-|-|-|
|32|1047430 Bytes|0.81315|
|16|522958 Bytes|0.81347|
|8|268472 Bytes|0.80791|
|7|268472 Bytes|0.80791|

## Byte Cost
根據[torch的官方手冊](https://pytorch.org/docs/stable/tensors.html)，我們知道 torch.FloatTensor 預設是 32-bit，也就是佔了 4byte 的空間，而 FloatTensor 系列最低可以容忍的是 16-bit。

為了方便操作，我們之後會將 state_dict 轉成 numpy array，因此我們可以先看看 numpy 有甚麼樣的 type 可以使用。

| | | |
|-|-|-|
|16|half precision float| 符号 bit + 5 bits 指数 + 10 bits 尾数|
|32|singal precision float| 符号 bit + 8 bits 指数 + 23 bits 尾数|
|64|double precision float| 符号 bit + 11 bits 指数 + 52 bits 尾数|

而我們發現 numpy 最低有 float16 可以使用，因此我們可以直接靠轉型將 32-bit 的 tensor 轉換成 16-bit 的 ndarray 存起來。

# Read state_dict

下載我們已經train好的小model的state_dict進行測試。

In [5]:
import os
import torch
path = "hw7_data/"

print(f"\noriginal cost: {os.stat(path + 'student_custom_small.bin').st_size} bytes.")
params = torch.load(path + 'student_custom_small.bin')


original cost: 1047430 bytes.


# 32-bit Tensor -> 16-bit

In [6]:
import numpy as np
import pickle

def encode16(params, file_name): # 將 params 壓縮成 16-bit 後輸出到 file_name
    custom_dict = {}
    for (name, param) in params.items():
        param = np.float64(param.cpu().numpy())
        if type(param) == np.ndarray: # 有些東西不屬於ndarray，只是一個數字，這個時候我們就不用壓縮。
            custom_dict[name] = np.float16(param)
        else:
            custom_dict[name] = param

    pickle.dump(custom_dict, open(file_name, 'wb'))


def decode16(file_name): # 從 file_name 讀取各個 params，將其從 16-bit 還原回 torch.tensor 後存進 state_dict 內
    params = pickle.load(open(file_name, 'rb'))
    custom_dict = {}
    for (name, param) in params.items():
        param = torch.tensor(param)
        custom_dict[name] = param

    return custom_dict


encode16(params, path + '16_bit_model.pkl')
print(f"16-bit cost: {os.stat(path + '16_bit_model.pkl').st_size} bytes.")

16-bit cost: 521550 bytes.
